# Amini Soil Prediction Challenge

#### Load required packages

In [441]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [442]:
# Load datasets
train_df = pd.read_csv('Train.csv')
train_gap_df = pd.read_csv('Gap_Train.csv')
test_df = pd.read_csv('Test.csv')
test_gap_df = pd.read_csv('Gap_Test.csv')

In [443]:
train_df.columns, train_gap_df.columns, test_gap_df.columns

(Index(['site', 'PID', 'lon', 'lat', 'pH', 'alb', 'bio1', 'bio12', 'bio15',
        'bio7', 'bp', 'cec20', 'dows', 'ecec20', 'hp20', 'ls', 'lstd', 'lstn',
        'mb1', 'mb2', 'mb3', 'mb7', 'mdem', 'para', 'parv', 'ph20', 'slope',
        'snd20', 'soc20', 'tim', 'wp', 'xhp20', 'BulkDensity', 'N', 'P', 'K',
        'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B'],
       dtype='object'),
 Index(['Nutrient', 'Required', 'Available', 'Gap', 'PID'], dtype='object'),
 Index(['Nutrient', 'Required', 'PID'], dtype='object'))

In [444]:
test_gap_df.head()

,Nutrient,Required,PID
0,N,100.0,ID_NGS9Bx
1,P,40.0,ID_NGS9Bx
2,K,52.0,ID_NGS9Bx
3,Ca,12.0,ID_NGS9Bx
4,Mg,8.0,ID_NGS9Bx


In [445]:
# Simulate dummy ppm values between 5 and 50 (you can adjust this range)
np.random.seed(42)  # For reproducibility
test_gap_df['ppm'] = np.random.uniform(5, 50, size=len(test_gap_df))

In [446]:
for col in test_gap_df.columns:
  print(col)

Nutrient
Required
PID
ppm


In [447]:
test_gap_df.head()

,Nutrient,Required,PID,ppm
0,N,100.0,ID_NGS9Bx,21.854305
1,P,40.0,ID_NGS9Bx,47.782144
2,K,52.0,ID_NGS9Bx,37.939727
3,Ca,12.0,ID_NGS9Bx,31.939632
4,Mg,8.0,ID_NGS9Bx,12.020839


In [448]:
# Constants
soil_depth = 20  # cm
bulk_density = 1.3  # g/cm³ (assumed)

# Calculate Available (kg/ha)
test_gap_df['Available'] = test_gap_df['ppm'] * soil_depth * bulk_density * 0.1

# Calculate Gap
test_gap_df['Gap'] = test_gap_df['Required'] - test_gap_df['Available']

print(test_gap_df)

      Nutrient  Required        PID        ppm   Available        Gap
0            N    100.00  ID_NGS9Bx  21.854305   56.821194  43.178806
1            P     40.00  ID_NGS9Bx  47.782144  124.233574 -84.233574
2            K     52.00  ID_NGS9Bx  37.939727   98.643291 -46.643291
3           Ca     12.00  ID_NGS9Bx  31.939632   83.043043 -71.043043
4           Mg      8.00  ID_NGS9Bx  12.020839   31.254181 -23.254181
...        ...       ...        ...        ...         ...        ...
26593       Fe      0.80  ID_oMn2Yb  37.798206   98.275335 -97.475335
26594       Mn      0.40  ID_oMn2Yb   5.382641   13.994868 -13.594868
26595       Zn      0.40  ID_oMn2Yb  14.928076   38.812997 -38.412997
26596       Cu      0.20  ID_oMn2Yb  27.135883   70.553296 -70.353296
26597        B      0.08  ID_oMn2Yb  23.901562   62.144061 -62.064061

[26598 rows x 6 columns]


In [449]:
test_df.columns, test_gap_df.columns

(Index(['site', 'PID', 'lon', 'lat', 'pH', 'alb', 'bio1', 'bio12', 'bio15',
        'bio7', 'bp', 'cec20', 'dows', 'ecec20', 'hp20', 'ls', 'lstd', 'lstn',
        'mb1', 'mb2', 'mb3', 'mb7', 'mdem', 'para', 'parv', 'ph20', 'slope',
        'snd20', 'soc20', 'tim', 'wp', 'xhp20', 'BulkDensity'],
       dtype='object'),
 Index(['Nutrient', 'Required', 'PID', 'ppm', 'Available', 'Gap'], dtype='object'))

### Earth observation data

In [450]:
landsat_df = pd.read_csv("LANDSAT8_data_updated.csv")

In [451]:
for col in landsat_df.columns:
  print(col)

QA_PIXEL
QA_RADSAT
SR_B1
SR_B2
SR_B3
SR_B4
SR_B5
SR_B6
SR_B7
ST_B10
date
lat
lon
PID


### Satellite Data.

In [452]:
s1 = pd.read_csv("Sentinel1_data.csv")
s2 = pd.read_csv("Sentinel2_data.csv")

In [453]:
for col in s1.columns:
  print(col)

VH
VV
date
instrumentMode
lat
lon
orbitProperties_pass
relativeOrbitNumber_start
PID


In [454]:
for col in s2.columns:
  print(col)

B1
B11
B12
B2
B3
B4
B5
B6
B7
B8
B8A
B9
CLOUDY_PIXEL_PERCENTAGE
MEAN_SOLAR_ZENITH_ANGLE
NODATA_PIXEL_PERCENTAGE
SENSING_ORBIT_NUMBER
SPACECRAFT_NAME
date
lat
lon
PID


### Check the shpe of test and train data

In [455]:
test_gap_df.head()

,Nutrient,Required,PID,ppm,Available,Gap
0,N,100.0,ID_NGS9Bx,21.854305,56.821194,43.178806
1,P,40.0,ID_NGS9Bx,47.782144,124.233574,-84.233574
2,K,52.0,ID_NGS9Bx,37.939727,98.643291,-46.643291
3,Ca,12.0,ID_NGS9Bx,31.939632,83.043043,-71.043043
4,Mg,8.0,ID_NGS9Bx,12.020839,31.254181,-23.254181


In [456]:
train_df.shape, train_gap_df.shape

((7744, 44), (85184, 5))

In [457]:
# Pivot train_gap_df so each PID has one row and each nutrient has its own column
gap_wide = train_gap_df.pivot(index="PID", columns="Nutrient", values="Gap")

# Rename columns to make them clear as gap targets
gap_wide.columns = [f"Gap_{col}" for col in gap_wide.columns]

# Reset index so PID is a column again (not the index)
gap_wide = gap_wide.reset_index()

# Merge gap values into train_df using PID
merged_train_df = train_df.merge(gap_wide, on="PID", how="left")

# Preview the result
merged_train_df.head()


,site,PID,lon,lat,pH,alb,bio1,bio12,bio15,bio7,...,Gap_Ca,Gap_Cu,Gap_Fe,Gap_K,Gap_Mg,Gap_Mn,Gap_N,Gap_P,Gap_S,Gap_Zn
0,site_id_bIEHwl,ID_I5RGjv,70.603761,46.173798,7.75,176,248,920,108,190,...,-19931.6,-8.5016,-218.784,-377.24,-6737.20,-247.80,-3696.0,39.0072,-4.5272,-1.9944
1,site_id_nGvnKc,ID_8jWzJ5,70.590479,46.078924,7.10,181,250,1080,113,191,...,-3575.2,-12.9328,-291.648,-407.04,-706.40,-1242.96,-4156.0,4.4320,-46.9760,-7.4128
2,site_id_nGvnKc,ID_UgzkN8,70.582553,46.048820,6.95,188,250,1109,111,191,...,-5506.8,-3.4208,-223.164,-388.92,-996.48,-189.40,-10120.0,-23.6560,-20.1200,-5.2940
3,site_id_nGvnKc,ID_DLLHM9,70.573267,46.021910,7.83,174,250,1149,112,191,...,-19701.6,-8.9168,-241.624,-542.96,-2120.24,-215.68,-6708.0,-78.1040,-32.1040,-14.1040
4,site_id_7SA9rO,ID_d009mj,70.585330,46.204336,8.07,188,250,869,114,191,...,-20980.4,-8.4658,-197.684,-205.40,-3309.60,-425.74,-2588.4,37.1400,-12.7676,-1.1730


In [458]:
merged_train_df.shape

(7744, 55)

In [459]:
list(merged_train_df.columns)

['site',
 'PID',
 'lon',
 'lat',
 'pH',
 'alb',
 'bio1',
 'bio12',
 'bio15',
 'bio7',
 'bp',
 'cec20',
 'dows',
 'ecec20',
 'hp20',
 'ls',
 'lstd',
 'lstn',
 'mb1',
 'mb2',
 'mb3',
 'mb7',
 'mdem',
 'para',
 'parv',
 'ph20',
 'slope',
 'snd20',
 'soc20',
 'tim',
 'wp',
 'xhp20',
 'BulkDensity',
 'N',
 'P',
 'K',
 'Ca',
 'Mg',
 'S',
 'Fe',
 'Mn',
 'Zn',
 'Cu',
 'B',
 'Gap_B',
 'Gap_Ca',
 'Gap_Cu',
 'Gap_Fe',
 'Gap_K',
 'Gap_Mg',
 'Gap_Mn',
 'Gap_N',
 'Gap_P',
 'Gap_S',
 'Gap_Zn']

#### Feature Selection

In [460]:
# Drop non-feature columns
drop_cols = ['site', 'PID'] + [col for col in train_df.columns if col.startswith('Gap_')] + ['N', 'P', 'K', 'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B']
new_train_data = merged_train_df.drop(columns=drop_cols)

In [461]:
missing_counts = new_train_data.isnull().sum()
print(missing_counts)

lon            0
lat            0
pH             0
alb            0
bio1           0
bio12          0
bio15          0
bio7           0
bp             0
cec20          0
dows           0
ecec20         5
hp20           5
ls             0
lstd           0
lstn           0
mb1            0
mb2            0
mb3            0
mb7            0
mdem           0
para           0
parv           0
ph20           0
slope          0
snd20          0
soc20          0
tim            0
wp             0
xhp20          5
BulkDensity    4
Gap_B          0
Gap_Ca         0
Gap_Cu         0
Gap_Fe         0
Gap_K          0
Gap_Mg         0
Gap_Mn         0
Gap_N          0
Gap_P          0
Gap_S          0
Gap_Zn         0
dtype: int64


In [462]:
for col in new_train_data.columns:
    n_missing = new_train_data[col].isnull().sum()
    print(f"{col}: {n_missing}")

lon: 0
lat: 0
pH: 0
alb: 0
bio1: 0
bio12: 0
bio15: 0
bio7: 0
bp: 0
cec20: 0
dows: 0
ecec20: 5
hp20: 5
ls: 0
lstd: 0
lstn: 0
mb1: 0
mb2: 0
mb3: 0
mb7: 0
mdem: 0
para: 0
parv: 0
ph20: 0
slope: 0
snd20: 0
soc20: 0
tim: 0
wp: 0
xhp20: 5
BulkDensity: 4
Gap_B: 0
Gap_Ca: 0
Gap_Cu: 0
Gap_Fe: 0
Gap_K: 0
Gap_Mg: 0
Gap_Mn: 0
Gap_N: 0
Gap_P: 0
Gap_S: 0
Gap_Zn: 0


In [463]:
from sklearn.impute import SimpleImputer

# Check missing counts
print("Missing before imputation:")
print(new_train_data.isnull().sum().sort_values(ascending=False).head(10))

# Impute ecec20, hp20, xhp20, BulkDensity with median
imputer = SimpleImputer(strategy='median')
cols_to_impute = ['ecec20', 'hp20', 'xhp20', 'BulkDensity']
new_train_data[cols_to_impute] = imputer.fit_transform(new_train_data[cols_to_impute])

# Verify no more missing
print("\nMissing after imputation:")
print(new_train_data[cols_to_impute].isnull().sum())

Missing before imputation:
hp20           5
ecec20         5
xhp20          5
BulkDensity    4
bio1           0
bio12          0
pH             0
alb            0
lon            0
lat            0
dtype: int64

Missing after imputation:
ecec20         0
hp20           0
xhp20          0
BulkDensity    0
dtype: int64


In [464]:
sum_missing = new_train_data.isnull().sum()
print(sum_missing)

lon            0
lat            0
pH             0
alb            0
bio1           0
bio12          0
bio15          0
bio7           0
bp             0
cec20          0
dows           0
ecec20         0
hp20           0
ls             0
lstd           0
lstn           0
mb1            0
mb2            0
mb3            0
mb7            0
mdem           0
para           0
parv           0
ph20           0
slope          0
snd20          0
soc20          0
tim            0
wp             0
xhp20          0
BulkDensity    0
Gap_B          0
Gap_Ca         0
Gap_Cu         0
Gap_Fe         0
Gap_K          0
Gap_Mg         0
Gap_Mn         0
Gap_N          0
Gap_P          0
Gap_S          0
Gap_Zn         0
dtype: int64


In [465]:
list(new_train_data.columns)

['lon',
 'lat',
 'pH',
 'alb',
 'bio1',
 'bio12',
 'bio15',
 'bio7',
 'bp',
 'cec20',
 'dows',
 'ecec20',
 'hp20',
 'ls',
 'lstd',
 'lstn',
 'mb1',
 'mb2',
 'mb3',
 'mb7',
 'mdem',
 'para',
 'parv',
 'ph20',
 'slope',
 'snd20',
 'soc20',
 'tim',
 'wp',
 'xhp20',
 'BulkDensity',
 'Gap_B',
 'Gap_Ca',
 'Gap_Cu',
 'Gap_Fe',
 'Gap_K',
 'Gap_Mg',
 'Gap_Mn',
 'Gap_N',
 'Gap_P',
 'Gap_S',
 'Gap_Zn']

In [466]:
gap_cols = [col for col in merged_train_df.columns if col.startswith("Gap_")]
nutrient_cols = ["N", "P", "K", "Ca", "Mg", "S", "Fe", "Mn", "Zn", "Cu", "B"]
drop_for_X = ["site", "PID"] + gap_cols + nutrient_cols

y = merged_train_df[[
  'Gap_B',
  'Gap_Ca',
  'Gap_Cu',
  'Gap_Fe',
  'Gap_K',
  'Gap_Mg',
  'Gap_Mn',
  'Gap_N',
  'Gap_P',
  'Gap_S',
  'Gap_Zn']]
X = merged_train_df.drop(columns=drop_for_X)


In [467]:
list(merged_train_df.columns)

['site',
 'PID',
 'lon',
 'lat',
 'pH',
 'alb',
 'bio1',
 'bio12',
 'bio15',
 'bio7',
 'bp',
 'cec20',
 'dows',
 'ecec20',
 'hp20',
 'ls',
 'lstd',
 'lstn',
 'mb1',
 'mb2',
 'mb3',
 'mb7',
 'mdem',
 'para',
 'parv',
 'ph20',
 'slope',
 'snd20',
 'soc20',
 'tim',
 'wp',
 'xhp20',
 'BulkDensity',
 'N',
 'P',
 'K',
 'Ca',
 'Mg',
 'S',
 'Fe',
 'Mn',
 'Zn',
 'Cu',
 'B',
 'Gap_B',
 'Gap_Ca',
 'Gap_Cu',
 'Gap_Fe',
 'Gap_K',
 'Gap_Mg',
 'Gap_Mn',
 'Gap_N',
 'Gap_P',
 'Gap_S',
 'Gap_Zn']

In [468]:
merged_train_df.shape, new_train_data.shape

((7744, 55), (7744, 42))

In [469]:
X.shape

(7744, 31)

In [470]:
merged_train_df.shape

(7744, 55)

In [471]:
for col in merged_train_df.columns:
  print(col)

site
PID
lon
lat
pH
alb
bio1
bio12
bio15
bio7
bp
cec20
dows
ecec20
hp20
ls
lstd
lstn
mb1
mb2
mb3
mb7
mdem
para
parv
ph20
slope
snd20
soc20
tim
wp
xhp20
BulkDensity
N
P
K
Ca
Mg
S
Fe
Mn
Zn
Cu
B
Gap_B
Gap_Ca
Gap_Cu
Gap_Fe
Gap_K
Gap_Mg
Gap_Mn
Gap_N
Gap_P
Gap_S
Gap_Zn


In [472]:
print("X columns (features):", X.columns.tolist())
print("X shape:", X.shape)
print("y columns (targets):", y.columns.tolist())
print("y shape:", y.shape)

X columns (features): ['lon', 'lat', 'pH', 'alb', 'bio1', 'bio12', 'bio15', 'bio7', 'bp', 'cec20', 'dows', 'ecec20', 'hp20', 'ls', 'lstd', 'lstn', 'mb1', 'mb2', 'mb3', 'mb7', 'mdem', 'para', 'parv', 'ph20', 'slope', 'snd20', 'soc20', 'tim', 'wp', 'xhp20', 'BulkDensity']
X shape: (7744, 31)
y columns (targets): ['Gap_B', 'Gap_Ca', 'Gap_Cu', 'Gap_Fe', 'Gap_K', 'Gap_Mg', 'Gap_Mn', 'Gap_N', 'Gap_P', 'Gap_S', 'Gap_Zn']
y shape: (7744, 11)


In [473]:
for col in X.columns:
  print(col)

lon
lat
pH
alb
bio1
bio12
bio15
bio7
bp
cec20
dows
ecec20
hp20
ls
lstd
lstn
mb1
mb2
mb3
mb7
mdem
para
parv
ph20
slope
snd20
soc20
tim
wp
xhp20
BulkDensity


## Split the data into Traning and validation sets

In [474]:
X.shape, merged_train_df.shape

((7744, 31), (7744, 55))

In [475]:
gap_cols = [col for col in merged_train_df.columns if col.startswith("Gap_")]
nutrient_cols = ["N", "P", "K", "Ca", "Mg", "S", "Fe", "Mn", "Zn", "Cu", "B"]
drop_for_X = ["site", "PID"] + gap_cols + nutrient_cols

y = merged_train_df[[
  'Gap_B',
  'Gap_Ca',
  'Gap_Cu',
  'Gap_Fe',
  'Gap_K',
  'Gap_Mg',
  'Gap_Mn',
  'Gap_N',
  'Gap_P',
  'Gap_S',
  'Gap_Zn']
    ]
X = merged_train_df.drop(columns=drop_for_X)

In [476]:
merged_train_df.shape

(7744, 55)

In [477]:
# from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.20, random_state=42
)

print("Training X:", X_train.shape, "Validation X:", X_val.shape)
print("Training y:", y_train.shape, "Validation y:", y_val.shape)

Training X: (6195, 31) Validation X: (1549, 31)
Training y: (6195, 11) Validation y: (1549, 11)


In [478]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error

# Instantiate a base regressor (e.g. RandomForest)
base_rf = RandomForestRegressor(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

# Wrap it in MultiOutputRegressor
multi_rf = MultiOutputRegressor(base_rf)

# Fit on the training split
multi_rf.fit(X_train, y_train)

# Predict on validation split
y_pred = multi_rf.predict(X_val)


In [479]:
y_pred

array([[-4.08968000e-01, -8.00812520e+03, -1.12746160e+01, ...,
         2.83615780e+01, -7.39210000e+00, -2.56543800e+00],
       [-4.15280000e-01, -6.15436600e+03, -7.37056200e+00, ...,
         3.54875320e+01, -1.61296860e+01, -3.70902600e+00],
       [-7.75042000e-01, -7.16090560e+03, -4.68186000e+00, ...,
         3.03805460e+01, -6.61416800e+00, -8.50802400e+00],
       ...,
       [-2.81318000e-01, -7.70483940e+03, -9.89101200e+00, ...,
         1.96788000e+01, -1.68225780e+01, -6.01798800e+00],
       [-8.17832000e-01, -1.51700388e+04, -8.39796800e+00, ...,
        -3.39419360e+01, -1.92229640e+01, -3.85002600e+00],
       [-7.21672000e-01, -7.55777940e+03, -4.55984200e+00, ...,
         1.44236580e+01, -1.44979060e+01, -1.25296040e+01]])

In [480]:
y_pred.shape

(1549, 11)

In [481]:
for col in X_val.columns:
  print(col)

lon
lat
pH
alb
bio1
bio12
bio15
bio7
bp
cec20
dows
ecec20
hp20
ls
lstd
lstn
mb1
mb2
mb3
mb7
mdem
para
parv
ph20
slope
snd20
soc20
tim
wp
xhp20
BulkDensity


In [482]:
y_pred.shape

(1549, 11)

In [483]:
from sklearn.metrics import mean_squared_error
import numpy as np

# After you’ve trained and predicted (y_pred is shape [n_val, 11])

for idx, nutrient in enumerate(y_train.columns):
    mse = mean_squared_error(
        y_val.iloc[:, idx],
        y_pred[:, idx]
    )
    rmse = np.sqrt(mse)
    print(f"{nutrient}:   RMSE = {rmse:.3f}")

# Overall RMSE across all nutrient gaps:
overall_mse = mean_squared_error(
    y_val.values.flatten(),
    y_pred.flatten()
)
overall_rmse = np.sqrt(overall_mse)
print(f"\nOverall RMSE (all gaps combined): {overall_rmse:.3f}")


Gap_B:   RMSE = 0.574
Gap_Ca:   RMSE = 3844.033
Gap_Cu:   RMSE = 12.753
Gap_Fe:   RMSE = 104.669
Gap_K:   RMSE = 505.897
Gap_Mg:   RMSE = 852.630
Gap_Mn:   RMSE = 135.136
Gap_N:   RMSE = 1222.015
Gap_P:   RMSE = 115.829
Gap_S:   RMSE = 41.526
Gap_Zn:   RMSE = 6.204

Overall RMSE (all gaps combined): 1253.989


### Retrain on Full Data(without splitting)

In [484]:
# Pivot train_gap_df so each PID has one row and each nutrient has its own column
gap_wide_test = test_gap_df.pivot(index="PID", columns="Nutrient", values="Gap")

In [485]:
gap_wide_test.columns = ['Gap_' + str(col) for col in gap_wide_test.columns]

In [486]:
for col in gap_wide_test.columns:
  print(col)

Gap_B
Gap_Ca
Gap_Cu
Gap_Fe
Gap_K
Gap_Mg
Gap_Mn
Gap_N
Gap_P
Gap_S
Gap_Zn


In [487]:
gap_wide_test.head()

,Gap_B,Gap_Ca,Gap_Cu,Gap_Fe,Gap_K,Gap_Mg,Gap_Mn,Gap_N,Gap_P,Gap_S,Gap_Zn
PID,,,,,,,,,,,
ID_002W8m,-119.339038,-97.291418,-14.778798,-59.608066,21.906702,-32.498662,-82.337101,43.087403,-46.330142,-77.316890,-25.171715
ID_00Mug5,-123.019319,-111.812631,-59.076090,-50.303008,-75.260719,-46.948324,-16.166541,29.284651,-31.307171,-17.447758,-15.029843
ID_046bTG,-89.200925,-50.647562,-23.268971,-58.863884,-61.856073,-112.135888,-100.395504,64.846316,-15.265407,-59.258752,-21.161487
ID_04cLhK,-97.489273,-27.362081,-113.436344,-77.863066,21.304121,-34.363099,-73.836737,46.107754,13.650793,-100.522079,-26.027958
ID_07jm5H,-22.060207,-98.197347,-66.863899,-78.530804,-70.155857,-63.419872,-81.389242,10.708299,-26.634418,-83.119182,-51.324200


In [488]:
test_gap_df.head()

,Nutrient,Required,PID,ppm,Available,Gap
0,N,100.0,ID_NGS9Bx,21.854305,56.821194,43.178806
1,P,40.0,ID_NGS9Bx,47.782144,124.233574,-84.233574
2,K,52.0,ID_NGS9Bx,37.939727,98.643291,-46.643291
3,Ca,12.0,ID_NGS9Bx,31.939632,83.043043,-71.043043
4,Mg,8.0,ID_NGS9Bx,12.020839,31.254181,-23.254181


In [489]:
# Reset index so PID is a column again (not the index)
gap_wide_test = gap_wide_test.reset_index()

In [490]:
gap_wide_test.head()

,PID,Gap_B,Gap_Ca,Gap_Cu,Gap_Fe,Gap_K,Gap_Mg,Gap_Mn,Gap_N,Gap_P,Gap_S,Gap_Zn
0,ID_002W8m,-119.339038,-97.291418,-14.778798,-59.608066,21.906702,-32.498662,-82.337101,43.087403,-46.330142,-77.316890,-25.171715
1,ID_00Mug5,-123.019319,-111.812631,-59.076090,-50.303008,-75.260719,-46.948324,-16.166541,29.284651,-31.307171,-17.447758,-15.029843
2,ID_046bTG,-89.200925,-50.647562,-23.268971,-58.863884,-61.856073,-112.135888,-100.395504,64.846316,-15.265407,-59.258752,-21.161487
3,ID_04cLhK,-97.489273,-27.362081,-113.436344,-77.863066,21.304121,-34.363099,-73.836737,46.107754,13.650793,-100.522079,-26.027958
4,ID_07jm5H,-22.060207,-98.197347,-66.863899,-78.530804,-70.155857,-63.419872,-81.389242,10.708299,-26.634418,-83.119182,-51.324200


In [491]:
# Merge gap values into train_df using PID
merged_test_df = test_df.merge(gap_wide_test, on="PID", how="left")

In [492]:
# Preview the result
merged_test_df.head()

,site,PID,lon,lat,pH,alb,bio1,bio12,bio15,bio7,...,Gap_Ca,Gap_Cu,Gap_Fe,Gap_K,Gap_Mg,Gap_Mn,Gap_N,Gap_P,Gap_S,Gap_Zn
0,site_id_hgJpkz,ID_NGS9Bx,69.170794,44.522885,6.86,144,256,910,108,186,...,-71.043043,-95.644492,-18.995783,-46.643291,-23.254181,-113.942609,43.178806,-84.233574,-19.251359,-82.930456
1,site_id_olmuI5,ID_YdVKXw,68.885265,44.741057,7.08,129,260,851,110,187,...,-22.273521,-84.386789,-73.596503,14.156324,-26.458328,-63.137567,-26.479453,-70.395789,-36.596342,-46.673809
2,site_id_PTZdJz,ID_MZAlfE,68.970210,44.675777,6.50,142,259,901,109,187,...,-92.865587,-32.751322,-81.512505,-14.360188,-28.361833,-18.034698,52.819076,-15.864336,-61.165429,-83.682748
3,site_id_DOTgr8,ID_GwCCMN,69.068751,44.647707,6.82,142,261,847,109,187,...,-36.639811,-16.823457,-63.697842,-55.582490,-16.427637,-26.878473,-24.019608,-85.978948,-81.055264,-70.535698
4,site_id_1rQNvy,ID_K8sowf,68.990002,44.577607,6.52,145,253,1109,110,186,...,-61.847958,-117.494800,-125.641401,2.529804,-68.965103,-103.290540,56.722742,-50.515107,-22.627971,-122.521376


In [493]:
merged_test_df.shape

(2418, 44)

In [494]:
test_gap_df.groupby("Nutrient")["Gap"].nunique()


,Gap
Nutrient,
B,2418
Ca,2418
Cu,2418
Fe,2418
K,2418
Mg,2418
Mn,2418
N,2418
P,2418


In [495]:
list(merged_test_df.columns)

['site',
 'PID',
 'lon',
 'lat',
 'pH',
 'alb',
 'bio1',
 'bio12',
 'bio15',
 'bio7',
 'bp',
 'cec20',
 'dows',
 'ecec20',
 'hp20',
 'ls',
 'lstd',
 'lstn',
 'mb1',
 'mb2',
 'mb3',
 'mb7',
 'mdem',
 'para',
 'parv',
 'ph20',
 'slope',
 'snd20',
 'soc20',
 'tim',
 'wp',
 'xhp20',
 'BulkDensity',
 'Gap_B',
 'Gap_Ca',
 'Gap_Cu',
 'Gap_Fe',
 'Gap_K',
 'Gap_Mg',
 'Gap_Mn',
 'Gap_N',
 'Gap_P',
 'Gap_S',
 'Gap_Zn']

In [496]:
gap_cols =['site', 'PID']+[col for col in merged_train_df.columns if col.startswith("Gap_")]
X_test = merged_test_df.drop(columns=gap_cols)

In [497]:
list(X_test.columns)

['lon',
 'lat',
 'pH',
 'alb',
 'bio1',
 'bio12',
 'bio15',
 'bio7',
 'bp',
 'cec20',
 'dows',
 'ecec20',
 'hp20',
 'ls',
 'lstd',
 'lstn',
 'mb1',
 'mb2',
 'mb3',
 'mb7',
 'mdem',
 'para',
 'parv',
 'ph20',
 'slope',
 'snd20',
 'soc20',
 'tim',
 'wp',
 'xhp20',
 'BulkDensity']

In [498]:
y_test = merged_test_df[['Gap_B',
 'Gap_Ca',
 'Gap_Cu',
 'Gap_Fe',
 'Gap_K',
 'Gap_Mg',
 'Gap_Mn',
 'Gap_N',
 'Gap_P',
 'Gap_S',
 'Gap_Zn']]

In [499]:
list(y_test.columns)

['Gap_B',
 'Gap_Ca',
 'Gap_Cu',
 'Gap_Fe',
 'Gap_K',
 'Gap_Mg',
 'Gap_Mn',
 'Gap_N',
 'Gap_P',
 'Gap_S',
 'Gap_Zn']

In [500]:
y_pred_test = multi_rf.predict(X_test)
y_pred_test.shape

(2418, 11)

In [501]:
y_pred_test

array([[-4.61552000e-01, -1.49188634e+04, -1.23966860e+01, ...,
        -4.30938400e+00, -1.12384140e+01, -4.02398200e+00],
       [-4.47146000e-01, -1.58701526e+04, -1.19214920e+01, ...,
         1.92184360e+01, -1.14938920e+01, -2.72681800e+00],
       [-4.95820000e-01, -1.33978820e+04, -1.19434140e+01, ...,
         3.22157160e+01, -9.87121400e+00, -3.72990800e+00],
       ...,
       [-5.95394000e-01, -3.13764640e+03, -2.87515000e+00, ...,
         1.83920380e+01, -1.38072100e+01, -1.91611560e+01],
       [-6.23366000e-01, -3.45627700e+03, -2.31155400e+00, ...,
         1.45287960e+01, -1.93869920e+01, -2.01997840e+01],
       [-1.17916600e+00, -6.37268540e+03, -4.64028600e+00, ...,
        -1.00788000e-01, -7.13054000e+00, -1.11589200e+01]])

In [502]:
# Convert preds_full into a DataFrame

col_names = y.columns.tolist()

preds_df = pd.DataFrame(y_pred_test, columns=col_names)

print(preds_df.columns)
print(preds_df.head())

Index(['Gap_B', 'Gap_Ca', 'Gap_Cu', 'Gap_Fe', 'Gap_K', 'Gap_Mg', 'Gap_Mn',
       'Gap_N', 'Gap_P', 'Gap_S', 'Gap_Zn'],
      dtype='object')
      Gap_B      Gap_Ca     Gap_Cu     Gap_Fe     Gap_K     Gap_Mg    Gap_Mn  \
0 -0.461552 -14918.8634 -12.396686 -324.33250 -353.5204 -4095.6538 -375.4548   
1 -0.447146 -15870.1526 -11.921492 -280.61944 -340.4522 -5464.0810 -382.0058   
2 -0.495820 -13397.8820 -11.943414 -341.20460 -357.8598 -4387.0202 -389.7280   
3 -0.506064 -14010.3576 -12.158618 -353.23946 -353.1620 -4506.4320 -385.2876   
4 -0.423098 -11576.7152  -9.824004 -309.40296 -414.6404 -3579.7242 -357.8694   

      Gap_N      Gap_P      Gap_S    Gap_Zn  
0 -4667.632  -4.309384 -11.238414 -4.023982  
1 -3043.810  19.218436 -11.493892 -2.726818  
2 -4860.562  32.215716  -9.871214 -3.729908  
3 -5006.726  33.992968 -10.115650 -4.571270  
4 -4465.954  25.966968  -8.457940 -4.193122  


In [503]:
pred_full_df = pd.DataFrame(y_pred_test, columns=y.columns)

In [504]:
pred_full_df.shape

(2418, 11)

In [505]:
list(pred_full_df.columns)

['Gap_B',
 'Gap_Ca',
 'Gap_Cu',
 'Gap_Fe',
 'Gap_K',
 'Gap_Mg',
 'Gap_Mn',
 'Gap_N',
 'Gap_P',
 'Gap_S',
 'Gap_Zn']

In [506]:
pred_full_df.head()

,Gap_B,Gap_Ca,Gap_Cu,Gap_Fe,Gap_K,Gap_Mg,Gap_Mn,Gap_N,Gap_P,Gap_S,Gap_Zn
0,-0.461552,-14918.8634,-12.396686,-324.33250,-353.5204,-4095.6538,-375.4548,-4667.632,-4.309384,-11.238414,-4.023982
1,-0.447146,-15870.1526,-11.921492,-280.61944,-340.4522,-5464.0810,-382.0058,-3043.810,19.218436,-11.493892,-2.726818
2,-0.495820,-13397.8820,-11.943414,-341.20460,-357.8598,-4387.0202,-389.7280,-4860.562,32.215716,-9.871214,-3.729908
3,-0.506064,-14010.3576,-12.158618,-353.23946,-353.1620,-4506.4320,-385.2876,-5006.726,33.992968,-10.115650,-4.571270
4,-0.423098,-11576.7152,-9.824004,-309.40296,-414.6404,-3579.7242,-357.8694,-4465.954,25.966968,-8.457940,-4.193122


In [507]:
pred_full_df['PID'] = merged_test_df['PID'].values

In [508]:
pred_full_df.shape, X_test.shape

((2418, 12), (2418, 31))

In [509]:
SampleSubmission_test = pred_full_df.melt(id_vars='PID',
                                     var_name='Nutrient',
                                     value_name='Gap')

In [510]:
SampleSubmission_test['PID'] = SampleSubmission_test['PID'].astype(str) + '_' + SampleSubmission_test['Nutrient'].str.replace('Gap_', '')


In [511]:
SampleSubmission_test = SampleSubmission_test[['PID', 'Gap']]

In [512]:
SampleSubmission_test.to_csv('SampleSubmission_test.csv', index=False)
print("✅ SampleSubmission_test.csv created (shape:", SampleSubmission_test.shape, ")")

✅ SampleSubmission_test.csv created (shape: (26598, 2) )


## Remote sensing data

In [513]:
import pandas as pd
file_names = {
    "MODIS_MOD16A2": "MODIS_MOD16A2_data.csv",
    "MODIS_MOD13Q1": "MODIS_MOD13Q1_data.csv",
    "MODIS_MCD43A4": "MODIS_MCD43A4_data.csv",
    "MODIS_MOD09GA": "MODIS_MOD09GA_data.csv",
    "MODIS_MOD11A1": "MODIS_MOD11A1_data.csv"
}
# Load all into a dictionary of DataFrames
modis_data = {name: pd.read_csv(path) for name, path in file_names.items()}


In [514]:
# Access individual datasets like this:
MODIS_MOD16A2 = modis_data["MODIS_MOD16A2"]
MODIS_MOD13Q1 = modis_data["MODIS_MOD13Q1"]
MODIS_MCD43A4 = modis_data["MODIS_MCD43A4"]
MODIS_MOD09GA = modis_data["MODIS_MOD09GA"]
MODIS_MOD11A1 = modis_data["MODIS_MOD11A1"]


In [515]:
MODIS_MOD16A2.shape, MODIS_MOD13Q1.shape

((935363, 6), (545563, 13))

In [516]:
MODIS_MOD09GA.shape, MODIS_MOD11A1.shape, MODIS_MCD43A4.shape

((7486447, 11), (2465791, 6), (7767381, 8))

In [517]:
# print column names
for name, df in modis_data.items():
    print(f"\n{name} columns:")
    print(df.columns.tolist())



MODIS_MOD16A2 columns:
['ET', 'PET', 'date', 'lat', 'lon', 'PID']

MODIS_MOD13Q1 columns:
['EVI', 'NDVI', 'RelativeAzimuth', 'SolarZenith', 'ViewZenith', 'date', 'lat', 'lon', 'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b07', 'PID']

MODIS_MCD43A4 columns:
['Nadir_Reflectance_Band1', 'Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band3', 'Nadir_Reflectance_Band4', 'date', 'lat', 'lon', 'PID']

MODIS_MOD09GA columns:
['date', 'lat', 'lon', 'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b04', 'sur_refl_b05', 'sur_refl_b06', 'sur_refl_b07', 'PID']

MODIS_MOD11A1 columns:
['LST_Day_1km', 'LST_Night_1km', 'date', 'lat', 'lon', 'PID']


In [518]:
# Create individual copies for each MODIS dataset
mod16_df = modis_data['MODIS_MOD16A2'].copy()
mod13_df = modis_data['MODIS_MOD13Q1'].copy()
mcd43_df = modis_data['MODIS_MCD43A4'].copy()
mod09_df = modis_data['MODIS_MOD09GA'].copy()
mod11_df = modis_data['MODIS_MOD11A1'].copy()


In [519]:
# Identify columns to average for each dataset
mod16_avg = mod16_df.groupby("PID")[["ET", "PET"]].mean().reset_index()

mod13_avg = mod13_df.groupby("PID")[[
    "EVI", "NDVI", "RelativeAzimuth", "SolarZenith", "ViewZenith",
    "sur_refl_b01", "sur_refl_b02", "sur_refl_b03", "sur_refl_b07"
]].mean().reset_index()

mod09_avg = mod09_df.groupby("PID")[[
    "sur_refl_b01", "sur_refl_b02", "sur_refl_b03", "sur_refl_b04",
    "sur_refl_b05", "sur_refl_b06", "sur_refl_b07"
]].mean().reset_index()

mod11_avg = mod11_df.groupby("PID")[["LST_Day_1km", "LST_Night_1km"]].mean().reset_index()

mcd43_avg = mcd43_df.groupby("PID")[[
    "Nadir_Reflectance_Band1", "Nadir_Reflectance_Band2",
    "Nadir_Reflectance_Band3", "Nadir_Reflectance_Band4"
]].mean().reset_index()

mod09ga_avg = mod09_df.groupby("PID")[[    "sur_refl_b01", "sur_refl_b02", "sur_refl_b03", "sur_refl_b04",
    "sur_refl_b05", "sur_refl_b06", "sur_refl_b07"]].mean().reset_index()


In [520]:
# Merge all five MODIS datasets on PID using outer join
modis_combined = mod09_avg.copy()

modis_combined = pd.merge(modis_combined, mod13_avg, on="PID", how="outer")
modis_combined = pd.merge(modis_combined, mod11_avg, on="PID", how="outer")
modis_combined = pd.merge(modis_combined, mcd43_avg, on="PID", how="outer")
modis_combined = pd.merge(modis_combined, mod09ga_avg, on="PID", how="outer")


In [521]:
final_data = pd.merge(merged_train_df, modis_combined, on="PID", how="left")
final_data.shape

(7744, 84)

In [522]:
list(final_data.columns)

['site',
 'PID',
 'lon',
 'lat',
 'pH',
 'alb',
 'bio1',
 'bio12',
 'bio15',
 'bio7',
 'bp',
 'cec20',
 'dows',
 'ecec20',
 'hp20',
 'ls',
 'lstd',
 'lstn',
 'mb1',
 'mb2',
 'mb3',
 'mb7',
 'mdem',
 'para',
 'parv',
 'ph20',
 'slope',
 'snd20',
 'soc20',
 'tim',
 'wp',
 'xhp20',
 'BulkDensity',
 'N',
 'P',
 'K',
 'Ca',
 'Mg',
 'S',
 'Fe',
 'Mn',
 'Zn',
 'Cu',
 'B',
 'Gap_B',
 'Gap_Ca',
 'Gap_Cu',
 'Gap_Fe',
 'Gap_K',
 'Gap_Mg',
 'Gap_Mn',
 'Gap_N',
 'Gap_P',
 'Gap_S',
 'Gap_Zn',
 'sur_refl_b01_x',
 'sur_refl_b02_x',
 'sur_refl_b03_x',
 'sur_refl_b04_x',
 'sur_refl_b05_x',
 'sur_refl_b06_x',
 'sur_refl_b07_x',
 'EVI',
 'NDVI',
 'RelativeAzimuth',
 'SolarZenith',
 'ViewZenith',
 'sur_refl_b01_y',
 'sur_refl_b02_y',
 'sur_refl_b03_y',
 'sur_refl_b07_y',
 'LST_Day_1km',
 'LST_Night_1km',
 'Nadir_Reflectance_Band1',
 'Nadir_Reflectance_Band2',
 'Nadir_Reflectance_Band3',
 'Nadir_Reflectance_Band4',
 'sur_refl_b01',
 'sur_refl_b02',
 'sur_refl_b03',
 'sur_refl_b04_y',
 'sur_refl_b05_y',
 'su

In [523]:
final_data[[ 'Gap_B',
 'Gap_Ca',
 'Gap_Cu',
 'Gap_Fe',
 'Gap_K',
 'Gap_Mg',
 'Gap_Mn',
 'Gap_N',
 'Gap_P',
 'Gap_S',
 'Gap_Zn']].head()


,Gap_B,Gap_Ca,Gap_Cu,Gap_Fe,Gap_K,Gap_Mg,Gap_Mn,Gap_N,Gap_P,Gap_S,Gap_Zn
0,-0.6208,-19931.6,-8.5016,-218.784,-377.24,-6737.20,-247.80,-3696.0,39.0072,-4.5272,-1.9944
1,-0.2240,-3575.2,-12.9328,-291.648,-407.04,-706.40,-1242.96,-4156.0,4.4320,-46.9760,-7.4128
2,-0.5624,-5506.8,-3.4208,-223.164,-388.92,-996.48,-189.40,-10120.0,-23.6560,-20.1200,-5.2940
3,-2.4952,-19701.6,-8.9168,-241.624,-542.96,-2120.24,-215.68,-6708.0,-78.1040,-32.1040,-14.1040
4,-0.8066,-20980.4,-8.4658,-197.684,-205.40,-3309.60,-425.74,-2588.4,37.1400,-12.7676,-1.1730


In [524]:
gap_cols =['site', 'PID']+[col for col in merged_train_df.columns if col.startswith("Gap_")]
X_test = final_data.drop(columns=gap_cols)

In [525]:
list(X_test.columns)

['lon',
 'lat',
 'pH',
 'alb',
 'bio1',
 'bio12',
 'bio15',
 'bio7',
 'bp',
 'cec20',
 'dows',
 'ecec20',
 'hp20',
 'ls',
 'lstd',
 'lstn',
 'mb1',
 'mb2',
 'mb3',
 'mb7',
 'mdem',
 'para',
 'parv',
 'ph20',
 'slope',
 'snd20',
 'soc20',
 'tim',
 'wp',
 'xhp20',
 'BulkDensity',
 'N',
 'P',
 'K',
 'Ca',
 'Mg',
 'S',
 'Fe',
 'Mn',
 'Zn',
 'Cu',
 'B',
 'sur_refl_b01_x',
 'sur_refl_b02_x',
 'sur_refl_b03_x',
 'sur_refl_b04_x',
 'sur_refl_b05_x',
 'sur_refl_b06_x',
 'sur_refl_b07_x',
 'EVI',
 'NDVI',
 'RelativeAzimuth',
 'SolarZenith',
 'ViewZenith',
 'sur_refl_b01_y',
 'sur_refl_b02_y',
 'sur_refl_b03_y',
 'sur_refl_b07_y',
 'LST_Day_1km',
 'LST_Night_1km',
 'Nadir_Reflectance_Band1',
 'Nadir_Reflectance_Band2',
 'Nadir_Reflectance_Band3',
 'Nadir_Reflectance_Band4',
 'sur_refl_b01',
 'sur_refl_b02',
 'sur_refl_b03',
 'sur_refl_b04_y',
 'sur_refl_b05_y',
 'sur_refl_b06_y',
 'sur_refl_b07']

In [526]:
gap_cols = [col for col in merged_train_df.columns if col.startswith("Gap_")]
nutrient_cols = ["N", "P", "K", "Ca", "Mg", "S", "Fe", "Mn", "Zn", "Cu", "B"]
drop_for_X = ["site", "PID"] + gap_cols + nutrient_cols

yy = final_data[[
  'Gap_B',
  'Gap_Ca',
  'Gap_Cu',
  'Gap_Fe',
  'Gap_K',
  'Gap_Mg',
  'Gap_Mn',
  'Gap_N',
  'Gap_P',
  'Gap_S',
  'Gap_Zn']
    ]
XX = final_data.drop(columns=drop_for_X + nutrient_cols)

In [527]:
list(XX.columns), list(yy.columns)

(['lon',
  'lat',
  'pH',
  'alb',
  'bio1',
  'bio12',
  'bio15',
  'bio7',
  'bp',
  'cec20',
  'dows',
  'ecec20',
  'hp20',
  'ls',
  'lstd',
  'lstn',
  'mb1',
  'mb2',
  'mb3',
  'mb7',
  'mdem',
  'para',
  'parv',
  'ph20',
  'slope',
  'snd20',
  'soc20',
  'tim',
  'wp',
  'xhp20',
  'BulkDensity',
  'sur_refl_b01_x',
  'sur_refl_b02_x',
  'sur_refl_b03_x',
  'sur_refl_b04_x',
  'sur_refl_b05_x',
  'sur_refl_b06_x',
  'sur_refl_b07_x',
  'EVI',
  'NDVI',
  'RelativeAzimuth',
  'SolarZenith',
  'ViewZenith',
  'sur_refl_b01_y',
  'sur_refl_b02_y',
  'sur_refl_b03_y',
  'sur_refl_b07_y',
  'LST_Day_1km',
  'LST_Night_1km',
  'Nadir_Reflectance_Band1',
  'Nadir_Reflectance_Band2',
  'Nadir_Reflectance_Band3',
  'Nadir_Reflectance_Band4',
  'sur_refl_b01',
  'sur_refl_b02',
  'sur_refl_b03',
  'sur_refl_b04_y',
  'sur_refl_b05_y',
  'sur_refl_b06_y',
  'sur_refl_b07'],
 ['Gap_B',
  'Gap_Ca',
  'Gap_Cu',
  'Gap_Fe',
  'Gap_K',
  'Gap_Mg',
  'Gap_Mn',
  'Gap_N',
  'Gap_P',
  'Gap_S

In [528]:
# from sklearn.model_selection import train_test_split

XX_train, XX_val, yy_train, yy_val = train_test_split(
    XX, yy, test_size=0.20, random_state=42
)

print("Training XX:", XX_train.shape, "Validation XX:", XX_val.shape)
print("Training yy:", yy_train.shape, "Validation yy:", yy_val.shape)

Training XX: (6195, 60) Validation XX: (1549, 60)
Training yy: (6195, 11) Validation yy: (1549, 11)


In [529]:
XX_train.shape, XX.shape

((6195, 60), (7744, 60))

In [530]:
list(XX_train.columns), list(XX.columns)

(['lon',
  'lat',
  'pH',
  'alb',
  'bio1',
  'bio12',
  'bio15',
  'bio7',
  'bp',
  'cec20',
  'dows',
  'ecec20',
  'hp20',
  'ls',
  'lstd',
  'lstn',
  'mb1',
  'mb2',
  'mb3',
  'mb7',
  'mdem',
  'para',
  'parv',
  'ph20',
  'slope',
  'snd20',
  'soc20',
  'tim',
  'wp',
  'xhp20',
  'BulkDensity',
  'sur_refl_b01_x',
  'sur_refl_b02_x',
  'sur_refl_b03_x',
  'sur_refl_b04_x',
  'sur_refl_b05_x',
  'sur_refl_b06_x',
  'sur_refl_b07_x',
  'EVI',
  'NDVI',
  'RelativeAzimuth',
  'SolarZenith',
  'ViewZenith',
  'sur_refl_b01_y',
  'sur_refl_b02_y',
  'sur_refl_b03_y',
  'sur_refl_b07_y',
  'LST_Day_1km',
  'LST_Night_1km',
  'Nadir_Reflectance_Band1',
  'Nadir_Reflectance_Band2',
  'Nadir_Reflectance_Band3',
  'Nadir_Reflectance_Band4',
  'sur_refl_b01',
  'sur_refl_b02',
  'sur_refl_b03',
  'sur_refl_b04_y',
  'sur_refl_b05_y',
  'sur_refl_b06_y',
  'sur_refl_b07'],
 ['lon',
  'lat',
  'pH',
  'alb',
  'bio1',
  'bio12',
  'bio15',
  'bio7',
  'bp',
  'cec20',
  'dows',
  'ece

In [531]:
list(XX.columns), list(final_data.columns)

(['lon',
  'lat',
  'pH',
  'alb',
  'bio1',
  'bio12',
  'bio15',
  'bio7',
  'bp',
  'cec20',
  'dows',
  'ecec20',
  'hp20',
  'ls',
  'lstd',
  'lstn',
  'mb1',
  'mb2',
  'mb3',
  'mb7',
  'mdem',
  'para',
  'parv',
  'ph20',
  'slope',
  'snd20',
  'soc20',
  'tim',
  'wp',
  'xhp20',
  'BulkDensity',
  'sur_refl_b01_x',
  'sur_refl_b02_x',
  'sur_refl_b03_x',
  'sur_refl_b04_x',
  'sur_refl_b05_x',
  'sur_refl_b06_x',
  'sur_refl_b07_x',
  'EVI',
  'NDVI',
  'RelativeAzimuth',
  'SolarZenith',
  'ViewZenith',
  'sur_refl_b01_y',
  'sur_refl_b02_y',
  'sur_refl_b03_y',
  'sur_refl_b07_y',
  'LST_Day_1km',
  'LST_Night_1km',
  'Nadir_Reflectance_Band1',
  'Nadir_Reflectance_Band2',
  'Nadir_Reflectance_Band3',
  'Nadir_Reflectance_Band4',
  'sur_refl_b01',
  'sur_refl_b02',
  'sur_refl_b03',
  'sur_refl_b04_y',
  'sur_refl_b05_y',
  'sur_refl_b06_y',
  'sur_refl_b07'],
 ['site',
  'PID',
  'lon',
  'lat',
  'pH',
  'alb',
  'bio1',
  'bio12',
  'bio15',
  'bio7',
  'bp',
  'cec20

In [532]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error

# Instantiate a base regressor (e.g. RandomForest)
base_rf = RandomForestRegressor(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

# Wrap it in MultiOutputRegressor
multi_rf = MultiOutputRegressor(base_rf)

# Fit on the training split
multi_rf.fit(XX_train, yy_train)

# Predict on validation split
yy_pred = multi_rf.predict(XX_val)


In [533]:
list(XX_train.columns)

['lon',
 'lat',
 'pH',
 'alb',
 'bio1',
 'bio12',
 'bio15',
 'bio7',
 'bp',
 'cec20',
 'dows',
 'ecec20',
 'hp20',
 'ls',
 'lstd',
 'lstn',
 'mb1',
 'mb2',
 'mb3',
 'mb7',
 'mdem',
 'para',
 'parv',
 'ph20',
 'slope',
 'snd20',
 'soc20',
 'tim',
 'wp',
 'xhp20',
 'BulkDensity',
 'sur_refl_b01_x',
 'sur_refl_b02_x',
 'sur_refl_b03_x',
 'sur_refl_b04_x',
 'sur_refl_b05_x',
 'sur_refl_b06_x',
 'sur_refl_b07_x',
 'EVI',
 'NDVI',
 'RelativeAzimuth',
 'SolarZenith',
 'ViewZenith',
 'sur_refl_b01_y',
 'sur_refl_b02_y',
 'sur_refl_b03_y',
 'sur_refl_b07_y',
 'LST_Day_1km',
 'LST_Night_1km',
 'Nadir_Reflectance_Band1',
 'Nadir_Reflectance_Band2',
 'Nadir_Reflectance_Band3',
 'Nadir_Reflectance_Band4',
 'sur_refl_b01',
 'sur_refl_b02',
 'sur_refl_b03',
 'sur_refl_b04_y',
 'sur_refl_b05_y',
 'sur_refl_b06_y',
 'sur_refl_b07']

In [534]:
from sklearn.metrics import mean_squared_error
import numpy as np

# After you’ve trained and predicted (y_pred is shape [n_val, 11])

for idx, nutrient in enumerate(yy_train.columns):
    mse = mean_squared_error(
        yy_val.iloc[:, idx],
        yy_pred[:, idx]
    )
    rmse = np.sqrt(mse)
    print(f"{nutrient}:   RMSE = {rmse:.3f}")

# Overall RMSE across all nutrient gaps:
overall_mse = mean_squared_error(
    yy_val.values.flatten(),
    yy_pred.flatten()
)
overall_rmse = np.sqrt(overall_mse)
print(f"\nOverall RMSE (all gaps combined): {overall_rmse:.3f}")

Gap_B:   RMSE = 0.578
Gap_Ca:   RMSE = 3907.171
Gap_Cu:   RMSE = 12.775
Gap_Fe:   RMSE = 106.507
Gap_K:   RMSE = 500.642
Gap_Mg:   RMSE = 859.460
Gap_Mn:   RMSE = 137.411
Gap_N:   RMSE = 1245.763
Gap_P:   RMSE = 116.548
Gap_S:   RMSE = 42.384
Gap_Zn:   RMSE = 7.586

Overall RMSE (all gaps combined): 1273.972


In [535]:
yy_test = final_data[['Gap_B',
 'Gap_Ca',
 'Gap_Cu',
 'Gap_Fe',
 'Gap_K',
 'Gap_Mg',
 'Gap_Mn',
 'Gap_N',
 'Gap_P',
 'Gap_S',
 'Gap_Zn']]

In [536]:
gap_cols =['site', 'PID']+[col for col in final_data.columns if col.startswith("Gap_")]
nutrient_cols = ["N", "P", "K", "Ca", "Mg", "S", "Fe", "Mn", "Zn", "Cu", "B"]
XX_test = final_data.drop(columns=gap_cols+nutrient_cols)

In [537]:
list(yy_test.columns), list(XX_test.columns)

(['Gap_B',
  'Gap_Ca',
  'Gap_Cu',
  'Gap_Fe',
  'Gap_K',
  'Gap_Mg',
  'Gap_Mn',
  'Gap_N',
  'Gap_P',
  'Gap_S',
  'Gap_Zn'],
 ['lon',
  'lat',
  'pH',
  'alb',
  'bio1',
  'bio12',
  'bio15',
  'bio7',
  'bp',
  'cec20',
  'dows',
  'ecec20',
  'hp20',
  'ls',
  'lstd',
  'lstn',
  'mb1',
  'mb2',
  'mb3',
  'mb7',
  'mdem',
  'para',
  'parv',
  'ph20',
  'slope',
  'snd20',
  'soc20',
  'tim',
  'wp',
  'xhp20',
  'BulkDensity',
  'sur_refl_b01_x',
  'sur_refl_b02_x',
  'sur_refl_b03_x',
  'sur_refl_b04_x',
  'sur_refl_b05_x',
  'sur_refl_b06_x',
  'sur_refl_b07_x',
  'EVI',
  'NDVI',
  'RelativeAzimuth',
  'SolarZenith',
  'ViewZenith',
  'sur_refl_b01_y',
  'sur_refl_b02_y',
  'sur_refl_b03_y',
  'sur_refl_b07_y',
  'LST_Day_1km',
  'LST_Night_1km',
  'Nadir_Reflectance_Band1',
  'Nadir_Reflectance_Band2',
  'Nadir_Reflectance_Band3',
  'Nadir_Reflectance_Band4',
  'sur_refl_b01',
  'sur_refl_b02',
  'sur_refl_b03',
  'sur_refl_b04_y',
  'sur_refl_b05_y',
  'sur_refl_b06_y',
  '

In [538]:
# nutrient_cols = ["N", "P", "K", "Ca", "Mg", "S", "Fe", "Mn", "Zn", "Cu", "B"]
# XX_test = XX_test.drop(columns=nutrient_cols)
yy_pred = multi_rf.predict(XX_test)
yy_pred.shape

(7744, 11)

In [539]:
nutrient_cols

['N', 'P', 'K', 'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B']

In [540]:
list(XX_test.columns),

(['lon',
  'lat',
  'pH',
  'alb',
  'bio1',
  'bio12',
  'bio15',
  'bio7',
  'bp',
  'cec20',
  'dows',
  'ecec20',
  'hp20',
  'ls',
  'lstd',
  'lstn',
  'mb1',
  'mb2',
  'mb3',
  'mb7',
  'mdem',
  'para',
  'parv',
  'ph20',
  'slope',
  'snd20',
  'soc20',
  'tim',
  'wp',
  'xhp20',
  'BulkDensity',
  'sur_refl_b01_x',
  'sur_refl_b02_x',
  'sur_refl_b03_x',
  'sur_refl_b04_x',
  'sur_refl_b05_x',
  'sur_refl_b06_x',
  'sur_refl_b07_x',
  'EVI',
  'NDVI',
  'RelativeAzimuth',
  'SolarZenith',
  'ViewZenith',
  'sur_refl_b01_y',
  'sur_refl_b02_y',
  'sur_refl_b03_y',
  'sur_refl_b07_y',
  'LST_Day_1km',
  'LST_Night_1km',
  'Nadir_Reflectance_Band1',
  'Nadir_Reflectance_Band2',
  'Nadir_Reflectance_Band3',
  'Nadir_Reflectance_Band4',
  'sur_refl_b01',
  'sur_refl_b02',
  'sur_refl_b03',
  'sur_refl_b04_y',
  'sur_refl_b05_y',
  'sur_refl_b06_y',
  'sur_refl_b07'],)

In [541]:
yy_pred

array([[-1.27931200e+00, -2.82599188e+04, -9.45526200e+00, ...,
        -9.36970280e+01, -1.82929664e+02, -8.82531800e+00],
       [-4.49386000e-01, -4.45957800e+03, -1.11663820e+01, ...,
        -1.38446140e+01, -3.82380920e+01, -6.80863200e+00],
       [-5.08422000e-01, -5.39953860e+03, -5.31623400e+00, ...,
        -2.49425820e+01, -1.99197800e+01, -5.58836400e+00],
       ...,
       [-5.55640000e-01, -3.79065860e+03, -5.64076800e+00, ...,
         3.32850060e+01, -8.79461600e+00, -1.93943740e+01],
       [-5.89244000e-01, -3.40484400e+03, -4.51476200e+00, ...,
         3.37549360e+01, -1.54014380e+01, -1.79216980e+01],
       [-3.81968000e-01, -3.46548940e+03, -4.68479000e+00, ...,
         3.45383140e+01, -1.61552580e+01, -1.74216120e+01]])

In [542]:
yy_pred.shape

(7744, 11)

In [543]:
# Convert preds_full into a DataFrame

col_names = yy.columns.tolist()

preds_data_df = pd.DataFrame(yy_pred, columns=col_names)

print(preds_data_df.columns)
print(preds_data_df.head())

Index(['Gap_B', 'Gap_Ca', 'Gap_Cu', 'Gap_Fe', 'Gap_K', 'Gap_Mg', 'Gap_Mn',
       'Gap_N', 'Gap_P', 'Gap_S', 'Gap_Zn'],
      dtype='object')
      Gap_B      Gap_Ca     Gap_Cu     Gap_Fe     Gap_K     Gap_Mg    Gap_Mn  \
0 -1.279312 -28259.9188  -9.455262 -211.76062 -389.6704 -1578.6874 -632.9614   
1 -0.449386  -4459.5780 -11.166382 -278.36386 -385.0058 -1034.4764 -950.1014   
2 -0.508422  -5399.5386  -5.316234 -226.82470 -449.0602 -1023.1116 -261.4168   
3 -1.945478 -17832.9372  -8.989072 -259.46496 -496.7400 -2725.0460 -272.5790   
4 -0.800368 -20441.5920  -9.312332 -215.95860 -207.4274 -2816.1924 -422.2360   

      Gap_N      Gap_P       Gap_S     Gap_Zn  
0 -3974.452 -93.697028 -182.929664  -8.825318  
1 -4073.634 -13.844614  -38.238092  -6.808632  
2 -7783.820 -24.942582  -19.919780  -5.588364  
3 -5774.788 -57.994760  -30.480634 -11.526316  
4 -2923.584  33.951278  -20.830588  -2.444222  


In [544]:
predictions_full_df = pd.DataFrame(yy_pred, columns=yy.columns)

In [545]:
predictions_full_df

,Gap_B,Gap_Ca,Gap_Cu,Gap_Fe,Gap_K,Gap_Mg,Gap_Mn,Gap_N,Gap_P,Gap_S,Gap_Zn
0,-1.279312,-28259.9188,-9.455262,-211.76062,-389.6704,-1578.6874,-632.96140,-3974.452,-93.697028,-182.929664,-8.825318
1,-0.449386,-4459.5780,-11.166382,-278.36386,-385.0058,-1034.4764,-950.10140,-4073.634,-13.844614,-38.238092,-6.808632
2,-0.508422,-5399.5386,-5.316234,-226.82470,-449.0602,-1023.1116,-261.41680,-7783.820,-24.942582,-19.919780,-5.588364
3,-1.945478,-17832.9372,-8.989072,-259.46496,-496.7400,-2725.0460,-272.57900,-5774.788,-57.994760,-30.480634,-11.526316
4,-0.800368,-20441.5920,-9.312332,-215.95860,-207.4274,-2816.1924,-422.23600,-2923.584,33.951278,-20.830588,-2.444222
...,...,...,...,...,...,...,...,...,...,...,...
7739,-0.676562,-4075.5536,-4.455376,-289.74480,-1061.9190,-629.9196,-404.36160,-4286.520,34.184318,-12.052808,-10.139410
7740,-0.472178,-3982.3344,-5.144000,-297.44244,-678.5652,-841.4586,-444.47820,-4142.432,33.532914,-13.392338,-9.938984
7741,-0.555640,-3790.6586,-5.640768,-307.00320,-788.8772,-530.7590,-458.47480,-4695.280,33.285006,-8.794616,-19.394374
7742,-0.589244,-3404.8440,-4.514762,-325.00060,-696.1918,-541.0828,-511.13624,-5187.960,33.754936,-15.401438,-17.921698


In [546]:
predictions_full_df['PID'] = final_data['PID'].values

In [547]:
list(predictions_full_df.columns)

['Gap_B',
 'Gap_Ca',
 'Gap_Cu',
 'Gap_Fe',
 'Gap_K',
 'Gap_Mg',
 'Gap_Mn',
 'Gap_N',
 'Gap_P',
 'Gap_S',
 'Gap_Zn',
 'PID']

In [548]:
TBS = predictions_full_df.melt(id_vars='PID',
                                     var_name='Nutrient',
                                     value_name='Gap')


In [549]:
TBS['PID'] = TBS['PID'].astype(str) + '_' + TBS['Nutrient'].str.replace('Gap_', '')

In [550]:
TBS = TBS[['PID', 'Gap']]

In [551]:
TBS.to_csv('TBS.csv', index=False)
print("✅ TBS.csv created (shape:", TBS.shape, ")")


✅ TBS.csv created (shape: (85184, 2) )
